*This notebook is intellectual property of Auquan and is distributed under the [Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International Public License](https://creativecommons.org/licenses/by-nc-nd/4.0/legalcode). Any modification or distribution of this notebook without express permission of Auquan is prohibited and will result in legal prosecution.*

# Confidence Intervals

We mentioned in the notebook on Expected Value and Standard Deviation that statistics derived from a sample (data available to us) may differ from true value (population statistic). For example, we want to measure the population mean, but we can only calculate a sample mean. We then want to use the sample mean to estimate the population mean. We use confidence intervals in an attempt to determine how accurately our sample mean estimates the population mean.

A confidence interval gives an estimated range of values between which the variable is likely to lie. This range is calculated from a given set of data or from a probability distribution
The selection of a confidence level for the interval determines the probability that the confidence interval will contain the value of the variable *over many computations*(read subtelty note below).
So, a 95% confidence interval for a variable states that the interval will contain the true population mean 95% of the time.

Consider the following questions:
- **What is your 95% confidence interval for the weight of heaviest mammal?**
- **What is your 95% confidence interval for the weight of your best friend?**

The range of your answers should be such that you think there’s only a 5% that the actual value lies outside that range. Therefore, for most, the range for the first question should be much wider than the second question. 
You may answer the first question as 2,000-200,000 kg, i.e.you believe there is a 95% chance the actual answer lie in this range. In other words, there is only a 5% chance that the heaviest mammal weighs less than 2,000kg or more than 200,000kg. Obviously you’re more confident about your best friend’s weight, your range may be 50-90kg*.

*(You could make this range 50-500kg, but that is not a true 95% interval, there’s less than 1% chance your best friend’s weight is outside that range)*

How can we construct these intervals mathematically and not just by intuition?

Let's say you want to estimate the average height of students in a university with 1000 students, you might do this by measuring 100 students and estimating that the mean of that sample was close to the population. Let's try that.

In [ ]:
import numpy as np

np.random.seed(100)

# Let's define some 'true' population parameters, we'll pretend we don't know these.
POPULATION_MU = 64
POPULATION_SIGMA = 5

# Generate our sample by drawing from the population distribution
sample_size = 100
heights = np.random.normal(POPULATION_MU, POPULATION_SIGMA, sample_size)
mean_height = np.mean(heights)
print('sample mean: %.2f'%mean_height)

Unfortunately simply reporting the sample mean doesn't do much for us, as we don't know how it relates to the population mean. To get a sense for how it might relate, we can look for how much variance there is in our sample. Higher variance indicates instability and uncertainty.

In [ ]:
print('sample standard deviation: %.2f'%np.std(heights))

This still doesn't help, to really get a sense of how our sample mean relates to the population mean we need to compute a standard error. The standard error is a measure of the variance of the sample mean.

**IMPORTANT**:  
**Computing a standard error involves assuming that the way you sample is unbaised, and that the data are normal and independent. If these conditions are violated, your standard error will be wrong. There are ways of testing for this and correcting.**

The formula for standard error is.

$$SE = \frac{\sigma}{\sqrt{n}}$$

Where $\sigma$ is the sample standard deviation and $n$ is the number of samples.

In [ ]:
SE = np.std(heights) / np.sqrt(sample_size)
print('standard error: %.2f'%SE)

Assuming our data are normally distributed, we can use the standard error to compute our confidence interval. 

To do this we set the desired confidence level (say 95%) and determine 95% of data lies within a range how many standard deviations of mean for our data's distribution. 

How does this work? 

Remember the Central Limit Theorem? If are samples are large enough (generally > 30 is taken as a threshold), the way we sample is unbaised, and the sampled are independent, CLT says that the distribution of mean of samples should follow a normal distribution.

The $\mu$ for this distribution will be the true population mean. The variance (and $\sigma$)
can be derived from the variance of a sum of independent random variables.

If $X_{1},X_{2},\ldots ,X_{n}$ are  $n$ independent observations from a population that has a mean $\mu_1$  and standard deviation $\sigma_1$ , then the variance of the sum $S=(X_{1}+X_{2}+\cdots +X_{n}$) is $n\sigma_1 ^{2}$.
The variance of $S/n$ (the sample mean ${\bar {x}}$) must be ${\frac {1}{n^{2}}}n\sigma_1 ^{2}={\frac {\sigma_1 ^{2}}{n}}$.
And the standard deviation of $S/n$ must be $\sigma_1 /{\sqrt {n}}$, the standard error.

From CLT, distribution of sample mean ${\bar {x}}$ should approximately be normal distribution. In a previous post, we mentioed that 95% of the observations in a normal dsitribution lie in a range $[-2\sigma,2\sigma]$ around the mean. Therefore, 95% times, our true population mean should lie between range $[-2SE,2SE]$ around calculated sample mean.


When the  the Central Limit Theorem applies and normality can be safely assumed; if sample sizes are smaller, a safer approach is to use a $t$-distribution with appropriately specified degrees of freedom. The actual way to compute the values is by using a [cumulative distribution function (CDF)](https://github.com/Auquan/Tutorials/blob/master/Random%20Variables.ipynb). If you need more background on Probability Distributions,  CDFs and inverse CDFs, read about them [here](https://en.wikipedia.org/wiki/Probability_density_function) and [here](https://en.wikipedia.org/wiki/Cumulative_distribution_function). Look [here](https://en.wikipedia.org/wiki/Student%27s_t-distribution) for information on the $t$-distribution. The numbers [2,-2] used for a normal dsitribution get replaced by appropriate t-value. We can check the 95% t-value number using one of the Python functions. 

**NOTE: Be careful when applying the Central Limit Theorem, however, as many datasets in finance are fundamentally non-normal and it is not safe to apply the theorem casually or without attention to subtlety.**

We can visualize the 95% mass bounds here.

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt

#Generate a normal distribution
x = np.linspace(-5,5,100)
y = stats.norm.pdf(x,0,1)
plt.plot(x,y)

# Plot the intervals
plt.vlines(-1.96, 0, 1, colors='r', linestyles='dashed')
plt.vlines(1.96, 0, 1, colors='r', linestyles='dashed')

fill_x = np.linspace(-1.96, 1.96, 500)
fill_y = stats.norm.pdf(fill_x, 0, 1)
plt.fill_between(fill_x, fill_y)

plt.xlabel('$\sigma$')
plt.ylabel('Normal PDF')
plt.show()

Now, rather than reporting our sample mean without any sense of the probability of it being correct, we can compute an interval and be much more confident that the population mean lies in that interval. To do this we take our sample mean $\mu$ and report $\left(\mu-2 SE , \mu+2SE\right)$.

This works because assuming normality, that interval will contain the population mean 95% of the time.

### SUBTLETY:
Note that it is incorrect to say that "The true mean lies in a range $\left(\mu-2 SE , \mu+2SE\right)$ with 95% probability," but unfortunately this is a very common misinterpretation. Rather, the 95% refers instead to the fact that over many computations of a 95% confidence interval, the true value will be in the interval in 95% of the cases (assuming correct calibration of the confidence interval, which we will discuss later). 

But in fact for a single sample and the single confidence interval computed from it, we have no way of assessing the probability that the interval contains the population mean. 

In the code below, we generate hundred 95% Confidence Intervals around a mean of 0. Notice how for some of them, the mean lies completely outside the interval

In [ ]:
np.random.seed(8309)
n = 100 # number of samples to take
samples = [np.random.normal(loc=0, scale=1, size=100) for _ in range(n)]

fig, ax = plt.subplots(figsize=(10, 7))
for i in np.arange(1, n, 1):
    sample_mean = np.mean(samples[i])  # calculate sample mean
    se = stats.sem(samples[i])  # calculate sample standard error
    sample_ci = [sample_mean - 1.96*se, sample_mean + 1.96*se]
    if ((sample_ci[0] <= 0) and (0 <= sample_ci[1])):
        plt.plot((sample_ci[0], sample_ci[1]), (i, i), color='blue', linewidth=1);
        plt.plot(np.mean(samples[i]), i, 'bo');
    else:
        plt.plot((sample_ci[0], sample_ci[1]), (i, i), color='red', linewidth=1);
        plt.plot(np.mean(samples[i]), i, 'ro');
plt.axvline(x=0, ymin=0, ymax=1, linestyle='--', label = 'Population Mean');
plt.legend(loc='best');
plt.title('100 95% Confidence Intervals for mean of 0')
plt.show()

Going back to our height's example, we can manually contruct confidence intervals using t-test

In [ ]:
# standard error SE was already calculated
t_val = stats.t.ppf((1+0.95)/2, 9)  # d.o.f. = 10 - 1
print('sample mean height: %.2f'%mean_height)
print('t-value: %.2f'%t_val) 
print('standard error: %.2f'%SE) 
print('confidence interval: [%.2f, %.2f]'%(mean_height - t_val * SE, mean_height + t_val * SE)) 

or use a built-in function in scipy.stats for computing the interval

In [ ]:
print('99% confidence interval: ', stats.t.interval(0.99, df = 9,loc=mean_height, scale=SE))
print('95% confidence interval: ', stats.t.interval(0.95, df = 9,loc=mean_height, scale=SE))
print('80% confidence interval: ', stats.t.interval(0.8, df = 9, loc=mean_height, scale=SE))

Note that as your confidence increases, the interval necessarily widens.

## What does this mean?

Confidence intervals allow us to set our desired confidence, and then report a range that will likely contain the population mean. The higher our desired confidence, the larger range we report. In general once can never report a single point value, because the probability that any given point is the true population mean is incredibly small. Let's see how our intervals tighten as we change sample size.

In [ ]:
np.random.seed(10)

sample_sizes = [10, 100, 1000]
for s in sample_sizes:
    heights = np.random.normal(POPULATION_MU, POPULATION_SIGMA, s)
    SE = np.std(heights) / np.sqrt(s)
    print('Sample Size: %i, Intervals:'%s, stats.norm.interval(0.95, loc=mean_height, scale=SE))

## Visualizing Confidence Intervals

Confidence Intervals are a very important concept, and we will make repeated use of them while building our strategies, so make sure you understand this well. We visualize what the intervals mean in relation to our sample height data below.

In [ ]:
sample_size = 100
heights = np.random.normal(POPULATION_MU, POPULATION_SIGMA, sample_size)
SE = np.std(heights) / np.sqrt(sample_size)
(l, u) = stats.norm.interval(0.95, loc=np.mean(heights), scale=SE)

print('%.2f,%.2f'%(l, u))

plt.hist(heights, bins=20)
plt.xlabel('Height')
plt.ylabel('Frequency')

# Just for plotting
y_height = 5
plt.plot([l, u], [y_height, y_height], '-', color='r', linewidth=4, label='Confidence Interval')
plt.plot(np.mean(heights), y_height, 'o', color='r', markersize=10)
plt.show()

**Ex 1: Calculate the 95% confidence interval for average returns of AAPL. Plot this interval on a histogram of AAPL returns to see what the intervals look like.**


In [ ]:
# Install yahoo finance to obtain historical market data
!pip install yfinance

In [ ]:
import yfinance as yf

# Download data for apple
startDateStr = '2012-12-31'
endDateStr = '2017-12-31'
instrumentId = 'AAPL'
data = yf.download(instrumentId, startDateStr, endDateStr)
price_aapl = data.Close

In [ ]:
## Calculate 95% interval

## Important: Violation of Assumptions

The computation of a standard deviation, standard error, and confidence interval all rely on certain assumptions. If these assumptions are violated then the 95% confidence interval will not necessarily contain the population parameter 95% of the time. Let's work with an example to verify this:

### Example: Autocorrelated Data

If your data generating process is autocorrelated(we will talk about autocorrelation in detail in the next sheet), then estimates of standard deviation will be wrong. This is because autocorrelated processes tend to produce more extreme values than normally distributed processes. In such a process, new values depend on previous values, so series that are already far from the mean are likely to stay far from the mean. 
$$X_t = a* X_{t-1} + \epsilon$$
$$\epsilon \sim \mathcal{N}(0,1)$$

Below, I have written a function generate desired samples of autocorrelated data. We also plot 100 samples to show what data looks like

In [ ]:
def generate_autocorrelated_data(theta, mu, sigma, N):
    # Initialize the array
    X = np.zeros((N, 1))
    
    for t in range(1, N):
        # X_t = theta * X_{t-1} + epsilon
        X[t] = theta * X[t-1] + np.random.normal(mu, sigma)
    return X

X = generate_autocorrelated_data(0.5, 0, 1, 100)

plt.plot(X)
plt.xlabel('t')
plt.ylabel('X[t]')
plt.show()

Now, we need to know the true population mean of this series. I hypothesize that for larger sample sizes, the sample mean should converge to zero. This is because the process is still centered around zero. Let's check this empirically. 

**Ex 2: Generate 200 sample means by varying the number of samples of data generated from 10 to 2000 and plot these means. Can you see convergence in sample means?**

In [ ]:
sample_means = np.zeros(200)

# Generate 200 sample means by varying sample data size between 10 and 2000 
# and calculating the mean of the sample
    
plt.bar(range(0, 200), sample_means);
plt.xlabel('Sample Size')
plt.ylabel('Sample Mean')
plt.show()

You should see some convergence as sample size increases. Now check what the mean of the sample means is.

In [ ]:
np.mean(sample_means)

Should be pretty close to zero. Let's assume that I've convinced you that the true population mean is 0. 

Now that we know the population mean, we want to check how accurate our confidence intervals are. Do this in the following manner:

**Ex 3:**  
**1. Write a function that can compute a 95% confidence interval for the mean of a sample data by calculating sample mean, sample standard deviation and standard error. You can use function `stats.norm.interval()` to generate actual bounds**  

**2. Write a second function that checks whether a given interval actually contains the true mean, 0. The function should return true when the computed bounds contain 0**

In [ ]:
def compute_confidence_interval(X):
    T = len(X)
    # Compute mu and sigma  and SE

    # Compute the bounds

    return bounds

# We'll make a function that returns true when the computed bounds contain 0
def check_interval_coverage(X):
    
    #calculate intervals using function above

    # Check to make sure if 0 is contained in the interval
    return

**3. Now run many trials(>=500): In each trial, sample data of fixed size, 100, compute a confidence interval, and then check if the confidence interval contains the population mean, 0. For what percentage of trials do the confidence intervals contain 0?**


In [ ]:
T = 100
trials = 500
times_correct = 0

# run 500 trials
    
print 'Empirical Coverage: ', times_correct/float(trials)

What did you get as the empirical coverage? Is that correct? 

What should the correct coverage be? The answer is the password for the final post, with a leading decimal and rounded to two decimals (if your answer is 0.735, the password is 0.74)

(In this case we'd need to do what's known as a [Newey-West correction](https://en.wikipedia.org/wiki/Newey%E2%80%93West_estimator) on our standard error estimate to account for the autocorrelation, but don't worry about this now)

**The point to drive home is that it's important to check for the assumptions you make. It is quick and easy to check if your data are stationary (which implies not autocorrelated), and it can save you a lot of pain and suffering to do so. A normality test such as `Jarque Bera` will also be a good idea, as it may detect certain distribution properties which may violate assumptions of many following statistical analyses.**

This presentation is for informational purposes only and does not constitute an offer to sell, a solicitation to buy, or a recommendation for any security; nor does it constitute an offer to provide investment advisory or other services by Auquan. Nothing contained herein constitutes investment advice or offers any opinion with respect to the suitability of any security, and any views expressed herein should not be taken as advice to buy, sell, or hold any security or as an endorsement of any security or company. In preparing the information contained herein, Auquan, has not taken into account the investment needs, objectives, and financial circumstances of any particular investor. Any views expressed and data illustrated herein were prepared based upon information, believed to be reliable, available to Auquan, at the time of publication. Auquan makes no guarantees as to their accuracy or completeness. All information is subject to change and may quickly become unreliable for various reasons, including changes in market conditions or economic circumstances.